In [1]:
import cv2
import numpy as np
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')
import matplotlib.pyplot as plt
from skimage import feature as ft
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from time import time
# #############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=1.0)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)


# #############################################################################
# Split into a training set and a test set using a stratified k fold

# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    lfw_people.images, y, test_size=0.25, random_state=42)
features=[]

for n_images in X_train:
    image_1=np.array(n_images,np.uint8)
    hog_feat = ft.hog(image_1,  # input image
                      pixels_per_cell=(6,6), # pixel per cell
                      cells_per_block=(2,2), # cells per blcok
                      block_norm = 'L1', #  block norm : str {‘L1’, ‘L1-sqrt’, ‘L2’, ‘L2-Hys’}
                      transform_sqrt = True, # power law compression (also known as gamma correction)
                      feature_vector=True, # flatten the final vectors
                      ) # return HOG map
    features.append(hog_feat)
t0=time()
print("train")
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf = clf.fit(features, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)
print(clf.n_splits_)


cv2.waitKey(0)
cv2.destroyAllWindows()
print("end")

Total dataset size:
n_samples: 1288
n_features: 11750
n_classes: 7


/home/lvfengkun/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


train
done in 1739.726s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
3
end


In [5]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
t0 = time()
features_1=[]
for n_images in X_test:
    image_1=np.array(n_images,np.uint8)
    hog_feat = ft.hog(image_1,  # input image
                      pixels_per_cell=(6,6), # pixel per cell
                      cells_per_block=(2,2), # cells per blcok
                      block_norm = 'L1', #  block norm : str {‘L1’, ‘L1-sqrt’, ‘L2’, ‘L2-Hys’}
                      transform_sqrt = True, # power law compression (also known as gamma correction)
                      feature_vector=True, # flatten the final vectors
                      ) # return HOG map
    features_1.append(hog_feat)
y_pred = clf.predict(features_1)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))


/home/lvfengkun/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


done in 5.503s
                   precision    recall  f1-score   support

     Ariel Sharon       1.00      0.85      0.92        13
     Colin Powell       0.94      1.00      0.97        60
  Donald Rumsfeld       0.92      0.81      0.86        27
    George W Bush       0.94      0.99      0.96       146
Gerhard Schroeder       0.92      0.96      0.94        25
      Hugo Chavez       1.00      0.67      0.80        15
       Tony Blair       0.94      0.83      0.88        36

      avg / total       0.94      0.94      0.94       322

[[ 11   0   1   1   0   0   0]
 [  0  60   0   0   0   0   0]
 [  0   1  22   2   1   0   1]
 [  0   0   1 145   0   0   0]
 [  0   0   0   1  24   0   0]
 [  0   1   0   3   0  10   1]
 [  0   2   0   3   1   0  30]]


In [6]:
import joblib
model_path = 'model/'
joblib.dump(clf, model_path + 'hog_model')

['model/hog_model']